# **DATA WRANGLING**
This file attemps to gather, collect, and transform all the raw datasets from the source attached below in order to analyse the data avilable and proceed with it. 
The following processes will be dealt with: 
1. Reading the data and transforming variables
2. Dealing with missing values
3. Data exploration
4. Reshaping data
5. Filtering data

Source: https://datos.madrid.es/sites/v/index.jsp?vgnextoid=374512b9ace9f310VgnVCM100000171f5a0aRCRD&buscar=true&Texto=BiciMAD&Sector=&Formato=&Periodicidad=&orderByCombo=CONTENT_INSTANCE_NAME_DECODE

#### **LIBRARIES**

In [2]:
import pandas as pd 
import json

#### **READ DATA**

**Situacion Estaciones BiciMAD**

In [17]:
# load data using Python JSON module
with open('../Data/Situacion Estaciones/201909.json','r') as f:
    data = json.loads(f.read())

sit_201909 = pd.json_normalize(
    data, 
    meta=['_id'],
    record_path =['stations']
)
sit_201909["_id"] = pd.to_datetime(sit_201909["_id"])
sit_201909.head()


,activate,name,reservations_count,light,total_bases,free_bases,number,longitude,no_available,address,latitude,dock_bikes,id,_id
0,1,Puerta del Sol A,0,0,24,17,1a,-3.7024255,0,Puerta del Sol nº 1,40.4168961,3,1,2019-09-01 00:10:32.593867
1,1,Puerta del Sol B,0,2,24,13,1b,-3.7024207,0,Puerta del Sol nº 1,40.4170009,10,2,2019-09-01 00:10:32.593867
2,1,Miguel Moya,0,2,24,8,2,-3.7058415,0,Calle Miguel Moya nº 1,40.4205886,16,3,2019-09-01 00:10:32.593867
3,1,Plaza Conde Suchil,0,0,18,12,3,-3.7069171,0,Plaza del Conde Suchil nº 2-4,40.4302937,3,4,2019-09-01 00:10:32.593867
4,1,Malasaña,0,2,24,8,4,-3.7025875,0,Calle Manuela Malasaña nº 5,40.4285524,12,5,2019-09-01 00:10:32.593867


Itineraios

In [3]:
itinerarios_list = ['it_201907', 'it_201908','it_201909', 'it_201910', 'it_201911', 'it_201912']

it_201909 = pd.read_json('../Data/Itinerarios/201909.json')
it_201909.head()

,_id,user_day_code,idplug_base,user_type,idunplug_base,travel_time,idunplug_station,ageRange,idplug_station,unplug_hourTime,zip_code
0,{'$oid': '5ee89f704299338f03784f6c'},77aa6ceada492a978e137a9d65264e4428a6075638a1ba...,21,1,9,207,14,5,17,2019-09-01T00:00:00Z,28015
1,{'$oid': '5ee89f704299338f03784f71'},70405121c0d8540baafd11e086f0bbcb7f8ca4d08f349d...,11,1,15,208,38,0,42,2019-09-01T00:00:00Z,
2,{'$oid': '5ee89f704299338f03784f78'},3495ba3768b648f59ed06a7da11a10375b3271ee01c98e...,17,1,4,279,104,4,75,2019-09-01T00:00:00Z,28053
3,{'$oid': '5ee89f714299338f03784f84'},7d9e21c5eb60b93ed161df402061a7b2fa9fa094f9cec9...,23,1,6,450,161,4,162,2019-09-01T00:00:00Z,
4,{'$oid': '5ee89f714299338f03784f88'},72d8e4bd0a23a16bdbd519223d36b5bbb3d792aecda32f...,4,1,11,457,99,0,12,2019-09-01T00:00:00Z,


Bases BiciMAD

In [11]:
bases = pd.read_excel('../Data/bases_bicimad.xls')
bases.head()

,Número,Gis_X,Gis_Y,Fecha de Alta,Distrito,Barrio,Calle,Nº Finca,Tipo de Reserva,Número de Plazas,Longitud,Latitud,Direccion
0,001 a,440443.61,4474290.65,43803,01 CENTRO,01-06 SOL,"ALCALA, CALLE, DE",2,BiciMAD,30,-3.701998,40.417111,"ALCALA, CALLE, DE, 2"
1,001 b,440480.56,4474301.74,43867,01 CENTRO,01-06 SOL,"ALCALA, CALLE, DE",6,BiciMAD,30,-3.701564,40.417213,"ALCALA, CALLE, DE, 6"
2,2,440134.83,4474678.23,41813,01 CENTRO,01-05 UNIVERSIDAD,"MIGUEL MOYA, CALLE, DE",1,BiciMAD,24,-3.705674,40.420580,"MIGUEL MOYA, CALLE, DE, 1"
3,3,440012.98,4475760.68,41813,07 CHAMBERÍ,07-02 ARAPILES,"CONDE DEL VALLE DE SUCHIL, PLAZA, DEL",2,BiciMAD,18,-3.707212,40.430322,"CONDE DEL VALLE DE SUCHIL, PLAZA, DEL, 2"
4,4,440396.40,4475565.36,41813,01 CENTRO,01-05 UNIVERSIDAD,"MANUELA MALASAÑA, CALLE, DE",3,BiciMAD,24,-3.702674,40.428590,"MANUELA MALASAÑA, CALLE, DE, 3"
